# Feature Engineering

Turn the training and test sets into feature matricies.

In [20]:
import datetime
import numpy as np
import pandas as pd
import re
from sklearn import preprocessing

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

Delete unneeded columns, and split the outcomes into separate numpy arrays.

In [21]:
le = preprocessing.LabelEncoder()
le.fit(train['OutcomeType'])
outcomes = le.transform(train['OutcomeType'])

train = train.drop(['AnimalID', 'OutcomeType', 'OutcomeSubtype'], axis = 1)
test = test.drop(['ID'], axis = 1)

Create the Sex, Intact/Neutered, and Age variables.

In [ ]:
def create_sex_variables(data):
    SexuponOutcome = data['SexuponOutcome'].fillna('Unknown')
    results = []
    for row in SexuponOutcome:
        row = row.split(' ')
        if len(row) == 1:
            row = ['Unknown', 'Unknown']
        results.append(row)
    NeuteredSprayed, Sex = zip(
        *[['Neutered', x[1]] if x[0] == 'Spayed' else x for x in results])
    return (data.assign(Neutered = NeuteredSprayed).assign(Sex = Sex)
            .drop(['SexuponOutcome'], axis = 1))